In [ ]:
import sys
from inference.YoloSamPipeline import YoloSam
import torch

In [ ]:
yolo_path = './yolov8-segm-0/yolov8n-seg/weights/best.pt'

device_id = 3
torch.cuda.set_device(device_id)

# the checkpoint and model_type (vit_h, vit_t, etc.) must be compatible
yolo_sam_pipeline = YoloSam(
    device=f'cuda:{device_id}', 
    yolo_checkpoint=yolo_path, 
    sam_checkpoint="./output_sam/ft_mobile_sam_final_2024-05-05 09:57:26.423152.pth", 
    # sam_checkpoint="./output_sam/ft_mobile_sam_final_2024-04-27 00:02:11.627528_last.pth", 
    model_type='vit_t',
    efficient_vit_enc=None,
    yolo_conf=0.2)

In [ ]:
masks = yolo_sam_pipeline.run_predict('../XMM_OM_dataset/zscaled_512_stretched/S0782961401_L.png', show_masks=True) 

In [ ]:
import os 

dir_files = os.listdir('../XMM_OM_dataset/zscaled_512_stretched')
inference_times = []
num_empty_images = 0.0
num_images = len(dir_files)
for file_ in dir_files:
    try:
        _, _, inf_time, empty_image = yolo_sam_pipeline.run_predict('../XMM_OM_dataset/zscaled_512_stretched/'+file_) 
        if empty_image==0:
            inference_times.append(inf_time)
        num_empty_images += empty_image
    except Exception as e:
        print(e)
        continue
        num_images -=1

In [ ]:
num_images-num_empty_images

In [ ]:
num_empty_images

In [ ]:
import numpy as np

np.mean(inference_times), np.std(inference_times)

In [ ]:
np.max(inference_times), np.min(inference_times)

In [ ]:
masks = yolo_sam_pipeline.run_predict('..//XMM_OM_dataset/zscaled_512_stretched/S0504550601_U.png') 
# masks = yolo_sam_pipeline.run_predict('./Euclid-Grism-mode-caption1-768x768.png') 

In [ ]:
import glob
from roboflow import Roboflow
from dataset import dataset_utils, voc_annotate_and_Roboflow_export

def export_image_det_to_Roboflow(input_dir, filename, masks, obj_results):
    class_names = obj_results[0].names
    class_labels = obj_results[0].boxes.data[:, -1].int().tolist()
    
    objects = []
    for i in range(len(masks)):
        # masks[i]: [ 1, H, W]
        mask_np = masks[i].detach().cpu().numpy()
        polygon = voc_annotate_and_Roboflow_export.binary_image_to_polygon(mask_np[0])
        bbox = dataset_utils.mask_to_bbox(mask_np)
        if class_names[class_labels[i]] != 'read-out-streak':
            objects.append({
                'name': class_names[class_labels[i]],
                'bbox': bbox,
                'segmentations': polygon[0]
            })
    if len(objects)>0:
        voc_annotate_and_Roboflow_export.create_annotation_SAM(
            filename=filename, 
            width=512, 
            height=512, 
            depth=3, 
            objects=objects, 
            offset=1.2) # generating xml file for VOC format
        image_path = input_dir+filename
        annotation_filename = filename.replace(".png", ".xml")
        upload_project.upload(image_path, annotation_filename, overwrite=True)
        os.remove(annotation_filename)
    else:
        print("No objects after label filtering.")

In [ ]:
import os
import torch
import cv2 
from dataset import dataset_utils, voc_annotate_and_Roboflow_export

# Optional Roboflow export in VOC format given filenames
export_to_Roboflow = False
import time

# best_model = mobile_sam_model.cpu()
if export_to_Roboflow:
    # Initialize Roboflow client
    rf = Roboflow(api_key="api_key")
    upload_project = rf.workspace("username").project("project") # error if the project doesn't exist

new_images_dir = '../XMM_OM_dataset/zscaled_512_stretched/'
new_image_files =  os.listdir(new_images_dir)
# best_model.eval()
    
with torch.no_grad(): 
    # eg_img = 'S0018141301_M.png'
    for image_name in new_image_files[200:1000]: # [3695:3730]
        print('Image', new_images_dir+image_name)
        image = cv2.imread(new_images_dir + image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        start_time = time.time()
        try:
            sam_mask_pre, obj_results, _ = yolo_sam_pipeline.run_predict(new_images_dir + image_name) 
            # if export_to_Roboflow:
                # export_image_det_to_Roboflow(new_images_dir, image_name, sam_mask_pre, obj_results)
                
        except Exception as e: # most likely the image had no annotations
            print(e)
            if export_to_Roboflow:
                upload_project.upload(new_images_dir+image_name)
            continue